In [2]:
import os
import time
from multiprocessing import Pool
from functools import partial
import pandas as pd
import json
import pyspark

from pyspark import SparkContext
from py4j.java_gateway import java_import
from pyspark.mllib.common import _to_java_object_rdd

In [3]:
# Import vnTokenizer from Java
java_import(sc._gateway.jvm, "vn.vitk.tok.Tokenizer")
Tokenizer = sc._jvm.vn.vitk.tok.Tokenizer
dataFolder = os.getcwd() + '/dat/tok'
token = Tokenizer(sc._jsc, dataFolder + "/lexicon.xml", dataFolder + "/regexp.txt")

# try multiprocess

In [6]:
# def tok(read_path, write_path, filename):
#     try:
#         with open(read_path+ '/'+ filename,'r') as fd:
#             json_data = json.load(fd)

#         title = json_data['title']
#         title = title.replace('\t', ' ')
#         title = title.replace('\n', ' ')

#         content = json_data['content']
#         content = content.replace('\t', ' ')
#         content = content.replace('\n', ' ')

#         title = ' '.join(title.split())
#         content = ' '.join(content.split())

#         if(content==title==""):
#             return

#         # write data
#         with open(write_path+ '/' + filename,'w') as fd:
#             title_tokened = token.tokenizeOneLine(title)
#             content_tokened = token.tokenizeOneLine(content)
#             fd.write(title_tokened)
#             fd.write(' ')
#             fd.write(content_tokened)
#     except:
#         bug_path = os.getcwd() + 'Data/log/except.txt' 
#         with open(bug_path,'a') as fd:
#             fd.write(read_path+ '/'+ filename)
#             fd.write('\n')

In [4]:
# topics = ['GiaoDuc', 'PhapLuat', 'TheGioi', 'TheThao', 'ThoiSu']
# topics = ['GiaoDuc']

In [5]:
# start = time.time()

# for topic in topics:

#     read_path = os.getcwd() + '/Data/Raw/' + topic
#     write_path = os.getcwd() + '/Data/Tokenized/' + topic
#     if(topic=='GiaoDuc'):
#         filenames = os.listdir(read_path)[1000:]
#     else:
#         filenames = os.listdir(read_path)
#     with Pool(4) as p:
#         p.map(partial(tok, read_path, write_path), filenames)

# end = time.time()
# print(end - start)

2017-10-23-13-39-53-732.txt
2017-10-23-13-36-54-6.txt
2017-10-23-13-37-02-430.txt
2017-10-23-13-40-18-318.txt
2017-10-23-13-40-46-930.txt
2017-10-23-13-40-47-371.txt
2017-10-23-13-41-54-426.txt
2017-10-23-13-44-06-670.txt
2017-10-23-13-43-36-715.txt
2017-10-23-13-42-16-575.txt
2017-10-23-13-43-38-162.txt
2017-10-23-13-42-26-145.txt
2017-10-23-13-42-28-207.txt
2017-10-23-13-42-51-795.txt
2017-10-23-13-48-08-903.txt
2017-10-23-13-49-48-649.txt
2017-10-23-13-46-10-883.txt
2017-10-23-13-50-29-943.txt
2017-10-23-13-51-36-833.txt
2017-10-23-13-50-31-283.txt
2017-10-23-13-50-23-770.txt
2017-10-23-13-53-19-683.txt
2017-10-23-14-07-45-390.txt
2017-10-23-13-53-04-617.txt
2360.536851167679


# without mp

In [32]:
topics = ['GiaoDuc', 'PhapLuat', 'TheGioi', 'TheThao', 'ThoiSu']
# topics = ['GiaoDuc']

In [ ]:
start = time.time()
for topic in topics:
    count = 0

    read_path = os.getcwd() + '/Data/Raw/' + topic
    write_path = os.getcwd() + '/Data/Tokenized/' + topic
    
    tmp_arr = []
    
    # resume
    list_tokened = os.listdir(write_path)
    list_raw = os.listdir(read_path)
    
    if(len(list_tokened)):
        last_file = list_tokened[-1]
        ind = list_tokened.index(last_file)
    else:
        ind = 0
        
    print(topic + ' Start at ' + str(ind))
    
    for filename in list_raw[ind:]:
        
#         try:
        
        # read data
        with open(read_path+ '/'+ filename,'r') as fd:
            json_data = json.load(fd)

        title = json_data['title']
        title = title.replace('\t', ' ')
        title = title.replace('\n', ' ')

        content = json_data['content']
        content = content.replace('\t', ' ')
        content = content.replace('\n', ' ')

        title = ' '.join(title.split())
        content = ' '.join(content.split())

        if(content==title==""):
            bug_path = os.getcwd() + '/Data/log/null.txt' 
            with open(bug_path,'a') as fd:
                fd.write(read_path+ '/'+ filename)
                fd.write('\n')
            continue
            
        # tokenize
        title_tokened = token.tokenizeOneLine(title)
        content_tokened = token.tokenizeOneLine(content)
        title_tokened += content_tokened
        
        tmp_arr.append((filename, title_tokened))
        
        count += 1
        
        if(count%100==0):
            # write data
            for fn, data in tmp_arr:
                with open(write_path+ '/' + fn,'w') as fd:
                    fd.write(data)
            tmp_arr = []
                
        if(count%1000==0):
            print(count, count*100/len(list_raw))
            end = time.time()
            print(end - start)
#         except:
#             bug_path = os.getcwd() + '/Data/log/except.txt' 
#             with open(bug_path,'a') as fd:
#                 fd.write(read_path+ '/'+ filename)
#                 fd.write('\n')
        

99
GiaoDuc Start at 99
1000 1.3345255094551132
2631.800597667694
2000 2.6690510189102263
5290.330135583878
3000 4.00357652836534
7928.733065128326


In [ ]:
start = time.time()
for topic in topics:
    count = 0

    read_path = os.getcwd() + '/Data/Raw/' + topic
    write_path = os.getcwd() + '/Data/Tokenized/' + topic

    for filename in os.listdir(read_path):
        try:
            # read data
            with open(read_path+ '/'+ filename,'r') as fd:
                json_data = json.load(fd)

            title = json_data['title']
            title = title.replace('\t', ' ')
            title = title.replace('\n', ' ')

            content = json_data['content']
            content = content.replace('\t', ' ')
            content = content.replace('\n', ' ')

            title = ' '.join(title.split())
            content = ' '.join(content.split())

            if(content==title==""):
                bug_path = os.getcwd() + '/Data/log/null.txt' 
                with open(bug_path,'a') as fd:
                    fd.write(read_path+ '/'+ filename)
                    fd.write('\n')
                continue

            # write data
            with open(write_path+ '/' + filename,'a') as fd:
                title_tokened = token.tokenizeOneLine(title)
                content_tokened = token.tokenizeOneLine(content)
                fd.write(title_tokened)
                fd.write(' ')
                fd.write(content_tokened)

            count += 1
            if(count%1000==0):
                print(count, count*100/len(os.listdir(read_path)))
                end = time.time()
                print(end - start)
        except:
            bug_path = os.getcwd() + '/Data/log/except.txt' 
            with open(bug_path,'a') as fd:
                fd.write(read_path+ '/'+ filename)
                fd.write('\n')
        

1000 1.3345255094551132
2946.1612083911896
2000 2.6690510189102263
5957.715993642807
3000 4.00357652836534
9872.391820907593
4000 5.338102037820453
13365.824478387833
5000 6.6726275472755665
16974.055755615234
6000 8.00715305673068
20596.21934556961
7000 9.341678566185793
24254.913489103317
8000 10.676204075640905
27952.09667277336
9000 12.01072958509602
31655.367478370667
